In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import random
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tag import StanfordNERTagger
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize


import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from nltk import word_tokenize, pos_tag
from nltk.tag import StanfordNERTagger
from sklearn.linear_model import RidgeClassifier
# from tweetokenize import Tokenizer


import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from nltk.tag.stanford import StanfordNERTagger
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer


In [2]:
venezuela_tweets = pd.read_csv('/Users/alessandro/Downloads/venezuela_201906_1_tweets_csv_hashed.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (19,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
venezuela_tweets.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'poll_choices'],
      dtype='object')

In [39]:
venezuela_tweets.shape

(569455, 31)

In [43]:
venezuela_tweets.userid.value_counts()

568620329                                       43231
801485278858854401                              39421
801200139155214336                              34912
801555575687495681                              33503
801463213833547776                              32870
751630413769113601                              31657
751635009052827648                              31073
881675624401833984                              27436
761401522798333952                              27209
892133369965944833                              26370
769380097359024128                              26136
768237282562310144                              26020
896121971989262336                              25472
778257194076831744                              22624
741092772233482241                              16400
801221525718335488                              12618
772513370599747585                              12099
801442896041873409                              12025
741112500356276225          

# Only English Words (so it doesn't crash my notebook)

In [4]:
tweets_mask = venezuela_tweets['is_retweet'] == False
tweets = venezuela_tweets[tweets_mask]

tweets_english = tweets[tweets['tweet_language'] == 'en']

In [36]:
data = tweets_english[['user_screen_name','tweet_text']]


In [37]:
data.user_screen_name

0                                       TRUMPTRAIN_17
1                                       Laureen4Trump
2                                       Laureen4Trump
3                                        EmmaTurnerBN
4                                      GODBLESSAMERIC
5                                      GODBLESSAMERIC
6                                      GODBLESSAMERIC
7                                      GODBLESSAMERIC
8                                      GODBLESSAMERIC
9                                      GODBLESSAMERIC
10                                     GODBLESSAMERIC
11                                     GODBLESSAMERIC
12                                    TrumpNewsDaily_
13                                    TrumpNewsDaily_
14                                    TrumpNewsDaily_
15                                    TrumpNewsDaily_
16                                    TrumpNewsDaily_
17                                    TrumpNewsDaily_
38                          

In [6]:
# data = data[:10000]

In [7]:
# len(data)

10000

# Corpus and Users built to loop through. Corpus will be X and Users Y.

In [8]:
def get_bag_of_words(tweet):
    tokenized = [ word for word in tweet.split() if not(word.startswith("http")) and not(word.startswith('#'))]
    return ' '.join(tokenized)



In [9]:
# def bag_of_words(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     text_list = [p.getText() for p in soup.find_all("p")]
#     sent_list = [sent.replace('\xa0', ' ').lower() for sent in text_list]
    
#     sent_isalnum = []
#     for i in range(len(sent_list)):
#         sent_isalnum.append(re.sub(r'[^a-z]+', ' ', sent_list[i]))
    
#     return ' '.join(sent_isalnum)

In [53]:
data['bag_of_words'] = data['tweet_text'].apply(get_bag_of_words)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
data.head()

,user_screen_name,tweet_text,bag_of_words
0,TRUMPTRAIN_17,Truth About “Right to Healthcare” Every Lib Ne...,Truth About “Right to Healthcare” Every Lib Ne...
1,Laureen4Trump,"BREAKING: Air Force Makes Tragic Announcement,...","BREAKING: Air Force Makes Tragic Announcement,..."
2,Laureen4Trump,HAPPENING NOW: Disney Facing a Scandal So Cata...,HAPPENING NOW: Disney Facing a Scandal So Cata...
3,EmmaTurnerBN,Another Stupid Gun Control Cartoon From the Le...,Another Stupid Gun Control Cartoon From the Left
4,GODBLESSAMERIC,➡️ 90 POUNDS OF COCAINE FOUND ON SHIP OWNED BY...,➡️ 90 POUNDS OF COCAINE FOUND ON SHIP OWNED BY...


# Looping through Corpus and appending each bag of words in a_tweets (List of Lists)
# Looping through Users and appending each user in a_user (List of Lists)

In [12]:
from nltk.tokenize import sent_tokenize

# corpus = data.bag_of_words
# # a_tweets= [corpus[0],corpus[1]]
# users = data.user_screen_name

In [13]:
# a_tweets = []
# for corp in corpus:
#     a_tweets.append(corp)


# a_user = []
# for user in users:
#     a_user.append(user)

# Making Tweets Enumerated so I can list comp through them and use string methods

In [14]:
import string

# a_tweets_lower = [[word.lower() for word in tweet]
#                  for tweet in a_tweets]

In [15]:
# from nltk.corpus import stopwords

# stopwords_ = set(stopwords.words('english'))
# print("--- stopwords in english: {}".format(stopwords_))

In [16]:
# import string

# punctuation_ = set(string.punctuation)
# print("--- punctuation: {}".format(string.punctuation))

# def filter_tokens(sent):
#     return([w for w in sent if not w in stopwords_ and not w in punctuation_])

# tweets_filtered = list(map(filter_tokens, a_tweets_lower))



In [17]:
from collections import Counter


# Term Frequency

In [18]:

# # term occurence = counting distinct words in each bag
# term_occ = list(map(lambda tweet : Counter(tweet), tweets_filtered))

# # term frequency = occurences over length of bag
# term_freq = list()
# for i in range(len(a_tweets)):
#     term_freq.append( {k: (v / float(len(tweets_filtered[i])))
#                        for k, v in term_occ[i].items()} )



# Obtaining document frequencies

In [19]:
# # document occurence = number of documents having this word
# # term frequency = occurences over length of bag

# doc_occ = Counter( [word for tweet in tweets_filtered for word in set(tweet)] )

# # document frequency = occurences over length of corpus
# doc_freq = {k: (v / float(len(tweets_filtered)))
#             for k, v in doc_occ.items()}

# # displaying vocabulary
# # print("\n--- full vocabulary: {}".format(doc_occ))
# # print("\n--- doc freq: {}".format(doc_freq))


In [20]:
# doc_freq

# Creating the vocabulary for indexing

In [21]:
# the minimum document frequency (in proportion of the length of the corpus)
# min_df = 0.0

# filtering items to obtain the vocabulary
# vocabulary = [ k for k,v in doc_freq.items()]

# original line
# vocabulary = [ k for k,v in doc_freq.items() if v >= min_df ]


In [22]:
# vocabulary = [ k for k,v in doc_freq.items()]


In [23]:
# vocabulary

# The TFIDF vector

In [55]:
tfidf = TfidfVectorizer(decode_error='replace', max_features=100)
document_tfidf_matrix = tfidf.fit_transform(data.bag_of_words)
# print(sorted(tfidf.vocabulary_))
print(document_tfidf_matrix.todense())

[[0.48681391 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.42975196 0.56383248 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [56]:
document_tfidf_matrix.todense().shape

(494275, 100)

In [57]:
df = pd.DataFrame(document_tfidf_matrix.todense(),columns=tfidf.vocabulary_)
df.head()

,about,right,to,breaking,makes,please,now,that,from,the,...,their,do,down,russia,rt,here,muslim,him,did,an
0,0.486814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.623244,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
X = document_tfidf_matrix
y = data.user_screen_name
print(X.shape)
print(y.shape)


(494275, 100)
(494275,)


In [61]:
X[:3,:10].todense()

matrix([[0.48681391, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]])

# This kernel dies which makes it so  i have to restart everything


In [28]:


# This kernel dies which makes it so everything i have to restart everything
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =.3)





# Try RandomForest

In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=2,
                             random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [31]:
y_pred = rf.predict(X_test)

In [32]:
(y_test == y_pred).mean()

0.08833333333333333

In [33]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

                                              precision    recall  f1-score   support

                                 AbbyMartinM       0.00      0.00      0.00       152
                               AlyssaNelsonR       0.00      0.00      0.00        66
                              America4Trump_       0.00      0.00      0.00       186
                             AnnabelleBakerF       0.00      0.00      0.00        64
                               AriaWilsonGOP       0.09      1.00      0.16       265
                             BreakingNewsDJT       0.00      0.00      0.00        17
                             CarolineWalkerB       0.00      0.00      0.00       227
                             Citizens4Trump_       0.00      0.00      0.00        19
                                DTrumpTrain_       0.00      0.00      0.00       136
 EAMY+7SRc0r53i1vfKV4UvVAc+DAI4rtEpUjFSbwbQ=       0.00      0.00      0.00        20
                                EmmaTurnerBN       0.

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
from sklearn.linear_model import LogisticRegressionCV

# Logistic Regression Classifier

In [35]:
lr = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial').fit(X_train, y_train)
lr.scores_

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase 

ValueError: could not broadcast input array from shape (10,29,10001) into shape (10)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
(y_pred == y_test).mean()

In [ ]:
print(classification_report(y_test,y_pred))

# Truncated SVD

# This kernel dies which makes it so everything i have to restart everything

In [30]:
from sklearn.decomposition import TruncatedSVD

# X = document_tfidf_matrix.toarray()
# svd = TruncatedSVD(n_components=3000, random_state=42)
# svd.fit(X)

Go back to lecture read everything.


Realistic Braulin

